<a href="https://colab.research.google.com/github/Frinhani/LearningMachineLearning/blob/main/classificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download pt_core_news_sm

In [12]:
import numpy as np
import spacy
import pandas as pd
from gensim.models import KeyedVectors

w2v_modelo_cbow = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Files/modelo_cbow.txt')
w2v_modelo_skipgram = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Files/modelo_skipgram.txt')
artigo_treino = pd.read_csv('/content/drive/MyDrive/Files/treino.csv')
artigo_teste = pd.read_csv('/content/drive/MyDrive/Files/teste.csv')

In [6]:
nlp = spacy.load('pt_core_news_sm', disable=['paser', 'ner', 'tagger', 'textcat'])
def tokenizador(texto):
  doc = nlp(texto)
  tokens_validos=[]
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text.lower())
  return tokens_validos

texto = 'Rio de Janeiro 124135326 *** *(((( é uma cidade maravilhosa!'
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [10]:
def combinacao_de_vetores_por_soma(palavras, modelo):
  
  vetor_resultante = np.zeros((1,300))
  
  for palavra in palavras:
    try:
      vetor_resultante += modelo.get_vector(palavra)
    except KeyError:
      pass

  return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[-1.32437672e+00  5.98350331e-01 -1.43126681e+00  3.19653052e+00
   3.77383024e-01  1.05998215e+00 -4.25641467e-01  1.85249817e+00
  -2.36823723e-01  1.73208658e+00  8.06936682e-01 -1.04468193e+00
  -1.46047980e+00 -2.48003477e+00 -4.11971387e-01 -1.35241293e+00
   1.07213159e+00 -5.01937333e-01  1.71996956e+00 -2.74857581e-02
   8.47721312e-01  1.67645764e+00 -1.48952062e+00  1.21011248e+00
   3.38149041e-01  1.76940392e+00 -3.60611498e-01  3.11516720e+00
   1.17610985e+00 -2.73023015e+00  1.35116425e+00  2.09008311e+00
   7.97988350e-01 -1.45457530e+00 -1.49714577e+00 -1.15652266e+00
  -4.11808304e-02 -1.79862797e-01  1.39708656e+00  3.34760293e-01
  -4.23425600e-01  6.23440287e-01 -2.80064832e-01 -7.63139889e-01
  -2.30080232e+00  9.93265305e-04  7.70332877e-01  7.94777919e-01
  -4.40640485e-01 -1.13231733e-01  1.34832121e+00  1.31866290e+00
   3.36287233e+00 -4.53738421e-02  6.12329364e-01 -7.92561308e-01
   4.61870462e-01 -3.63853663e-01 -5.09549320e-01  4.33169428e-01
 

In [13]:
def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))
  
  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)
  return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
  RL = LogisticRegression(max_iter = 800)
  RL.fit(x_treino,y_treino)
  categorias = RL.predict(x_teste)
  resultados = classification_report(y_teste, categorias)
  print(resultados)
  return RL

RL_cbow= classificador(w2v_modelo_cbow,
                       matriz_vetores_treino_cbow,
                       artigo_treino.category,
                       matriz_vetores_teste_cbow,
                       artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.76      6103
   cotidiano       0.63      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.83      0.22       131
     mercado       0.83      0.78      0.80      5867
       mundo       0.75      0.83      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.80      0.69     20513
weighted avg       0.82      0.79      0.80     20513



In [18]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_skipgram)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_skipgram)
print(matriz_vetores_treino_sg.shape)
print(matriz_vetores_treino_sg.shape)

RL_skip_gram= classificador(w2v_modelo_skipgram,
                       matriz_vetores_treino_sg,
                       artigo_treino.category,
                       matriz_vetores_teste_sg,
                       artigo_teste.category)

(90000, 300)
(90000, 300)
              precision    recall  f1-score   support

     colunas       0.81      0.72      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.14      0.86      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.76      0.84      0.80      2051

    accuracy                           0.80     20513
   macro avg       0.69      0.82      0.71     20513
weighted avg       0.82      0.80      0.81     20513



In [19]:
import pickle

In [20]:
with open('/content/drive/MyDrive/Files/rl_cbow.pkl', 'wb') as f:
  pickle.dump(RL_cbow, f)

with open('/content/drive/MyDrive/Files/rl_sg.pkl', 'wb') as f:
  pickle.dump(RL_skip_gram, f)